# 상관분석

* internal_metrics() 이상치를 한번 파악하려고 봤더니 상자그림이 이상하게 그려졌다. 25
* knob 이상치 제거할 필요 없다. 랜덤생성했기 때문에 14 (+ 범주형은 10개)

1. 데이터 전처리
2. 상관분석 함수
3. activedefrag 있는 경우
4. 전체 knob
5. activedefrag 없는 경우
6. pointbeir~

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
cluster0 =[ 
'used_memory_rss',
'used_memory_rss_human',
'rss_overhead_ratio',
'rss_overhead_bytes',
'mem_fragmentation_ratio',
'mem_fragmentation_bytes']

cluster1 = [
    'uptime_in_seconds',
    'hz',
    'lru_clock',
    'used_memory_peak',
    'maxmemory',
    'maxmemory_human',
    'rdb_last_save_time',
    'total_net_output_bytes',
    'used_cpu_sys',
    'used_cpu_user'
]

cluster2 = [
    'allocator_allocated',
    'allocator_active',
    'allocator_frag_bytes',
    'instantaneous_ops_per_sec',
    'instantaneous_input_kbps',
    'instantaneous_output_kbps'
]

cluster3 = [
    'allocator_resident',
    'allocator_rss_ratio',
    'allocator_rss_bytes'  
]

knob_list = [
    "save0_0",
    "save0_1",
    "save1_0",
    "save1_1",
    "save2_0",
    "save2_1",

    "rdbcompression",
    "rdbchecksum",
    "rdb-save-incremental-fsync",

    "activedefrag",
    "active-defrag-threshold-lower",
    "active-defrag-threshold-upper",
    "active-defrag-cycle-min",
    "active-defrag-cycle-max",
    
    "maxmemory",
    
    "maxmemory-policy",
    "maxmemory-samples",
    "lazyfree-lazy-eviction",
    "lazyfree-lazy-expire",
    "lazyfree-lazy-server-del",

    "hash-max-ziplist-entries", 
    "hash-max-ziplist-value",
    "activerehashing",
    "hz",
    "dynamic-hz"
]

activedefrag_knobs = [
    "activedefrag",
    "active-defrag-threshold-lower",
    "active-defrag-threshold-upper",
    "active-defrag-cycle-min",
    "active-defrag-cycle-max",
]

In [4]:
# 데이터 불러오기 internal metrics와 knob들의 상관관계분석
# internal_csv = pd.read_csv('./data/internal_metrcis0.csv')

file_dir = '../data/'
file_name = 'internal_metrics5.csv'

internal_csv = pd.read_csv(file_dir + file_name)

In [5]:
# 데이터 불러오기 knob_list
# knobs = pd.read_csv('./data/result_config0.csv')

file_dir = '../data/'
file_name = 'result_config5.csv'

knobs = pd.read_csv(file_dir + file_name)

In [6]:
# 컬럼 추출 함수
def extract_data(cluster_list, df):
    extracted_df = pd.DataFrame()
    for elem in cluster_list:
        extracted_df[elem] = df[elem]

    return extracted_df    

In [7]:
internal_csv['config_file'][0]

'/home/swredis/redis-sample-generation/configfile_act/config11001.conf'

## 데이터 전처리

In [8]:
# 클러스터 합치기
cluster = cluster0+cluster1+cluster2+cluster3

In [9]:
# 클러스터 요소에 해당되는 metrics 25개 추출
total_internal_df = extract_data(cluster, internal_csv)

In [10]:
# throughput 결측치 제거
total_internal_df.dropna()

,used_memory_rss,used_memory_rss_human,rss_overhead_ratio,rss_overhead_bytes,mem_fragmentation_ratio,mem_fragmentation_bytes,uptime_in_seconds,hz,lru_clock,used_memory_peak,...,used_cpu_user,allocator_allocated,allocator_active,allocator_frag_bytes,instantaneous_ops_per_sec,instantaneous_input_kbps,instantaneous_output_kbps,allocator_resident,allocator_rss_ratio,allocator_rss_bytes
0,42024960,40.08,1.00,-184320,1.09,3402352,28,11,3668563,38704616,...,4.884932,38655872,38920192,264320,16060,1121.38,79.90,42209280,1.08,3289088
1,42024960,40.08,1.00,-184320,1.09,3402352,28,34,3668604,38704632,...,4.937450,38658328,38932480,274152,28139,1964.82,139.72,42209280,1.08,3276800
2,41799680,39.86,0.99,-397312,1.08,3177072,28,3,3668639,38704616,...,4.774636,38655584,38920192,264608,39006,2723.66,193.47,42196992,1.08,3276800
3,42041344,40.09,1.00,-167936,1.09,3418736,28,26,3668673,38704616,...,5.038324,38655856,38920192,264336,16262,1135.51,81.10,42209280,1.08,3289088
4,42049536,40.10,0.95,-2260992,1.09,3426928,28,22,3668707,38704616,...,4.859807,38655744,38920192,264448,16797,1172.88,83.55,44310528,1.14,5390336
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1695,42020864,40.07,0.95,-2289664,1.09,3398256,28,22,3742679,38704616,...,4.959740,38656320,38920192,263872,22815,1593.08,113.37,44310528,1.14,5390336
1696,42016768,40.07,1.00,-192512,1.09,3394160,28,24,3742714,38704616,...,4.985621,38657928,38932480,274552,17227,1202.90,85.79,42209280,1.08,3276800
1697,42065920,40.12,0.95,-2244608,1.09,3443312,28,22,3742748,38704616,...,4.835025,38657928,38932480,274552,19414,1355.54,96.31,44310528,1.14,5378048
1698,42041344,40.09,0.95,-2269184,1.09,3336832,28,1,3742782,38704616,...,4.876490,38737672,39010304,272632,42905,2995.82,212.12,44310528,1.14,5300224


In [44]:
# internal_csv 결측치 확인 -> 없음!
total_internal_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1700 entries, 0 to 1699
Data columns (total 25 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   used_memory_rss            1700 non-null   int64  
 1   used_memory_rss_human      1700 non-null   float64
 2   rss_overhead_ratio         1700 non-null   float64
 3   rss_overhead_bytes         1700 non-null   int64  
 4   mem_fragmentation_ratio    1700 non-null   float64
 5   mem_fragmentation_bytes    1700 non-null   int64  
 6   uptime_in_seconds          1700 non-null   int64  
 7   hz                         1700 non-null   int64  
 8   lru_clock                  1700 non-null   int64  
 9   used_memory_peak           1700 non-null   int64  
 10  maxmemory                  1700 non-null   int64  
 11  maxmemory_human            1700 non-null   float64
 12  rdb_last_save_time         1700 non-null   int64  
 13  total_net_output_bytes     1700 non-null   int64

In [45]:
# maxmemory 처리 throughtput은 처리할 필요없다.
total_internal_df['maxmemory']

0       1887436800
1       2202009600
2       2831155200
3       1677721600
4       2516582400
           ...    
1695    2411724800
1696    2516582400
1697    3040870400
1698    2202009600
1699    1887436800
Name: maxmemory, Length: 1700, dtype: int64

In [46]:
# maxmemory 처리 knob
knobs['maxmemory'] = knobs['maxmemory'].str.strip('mb')  # 단위 자르기
knobs['maxmemory'] = pd.to_numeric(knobs['maxmemory'])  # 숫자형으로 바꾸기

### activedefrag 처리
* subTotal_knobs_Active : 약 1300개 이며 activedefrag가 있는 경우 
* subTotal_knobs_NoActive: 약 1300개 이며 activedefrag가 nan값으로 제거한 경우
* subTotal_knobs_T : 2500개 이며 activedefrag 상관없이 전체 데이터


* total_internal_Active : 약 1300개 이며 activedefrag가 있는 경우 
* total_internal_NoActive: 약 1300개 이며 activedefrag가 nan값으로 제거한 경우
* total_internal_T : 2500개 이며 activedefrag 상관없이 전체 데이터

In [47]:
# knob_list 만 추출, AOF 관련 제외
subTotal_knobs = extract_data(knob_list, knobs)

In [48]:
subTotal_knobs

,save0_0,save0_1,save1_0,save1_1,save2_0,save2_1,rdbcompression,rdbchecksum,rdb-save-incremental-fsync,activedefrag,...,maxmemory-policy,maxmemory-samples,lazyfree-lazy-eviction,lazyfree-lazy-expire,lazyfree-lazy-server-del,hash-max-ziplist-entries,hash-max-ziplist-value,activerehashing,hz,dynamic-hz
0,868,4,320,58,58,10285,no,yes,no,NaN,...,volatile-random,5,yes,yes,yes,506,144,no,20,yes
1,867,6,258,52,67,9984,no,no,no,NaN,...,allkeys-lfu,4,yes,no,yes,552,141,no,22,yes
2,911,6,319,54,65,9980,no,no,no,NaN,...,noeviction,5,yes,no,yes,552,185,yes,20,no
3,910,4,293,51,61,10016,yes,no,yes,yes,...,noeviction,5,yes,yes,no,459,120,no,19,no
4,859,5,276,46,59,10402,no,yes,no,yes,...,volatile-ttl,4,no,yes,yes,530,109,no,23,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1695,948,6,278,68,64,10339,no,yes,no,yes,...,allkeys-random,5,no,no,no,498,159,yes,22,yes
1696,874,6,349,41,65,9558,no,no,yes,yes,...,volatile-ttl,5,yes,yes,no,545,76,no,17,yes
1697,883,5,266,67,66,10176,yes,no,yes,NaN,...,volatile-random,5,yes,no,yes,493,124,yes,22,no
1698,844,5,368,59,54,10615,no,no,no,NaN,...,allkeys-random,5,yes,no,yes,452,154,yes,23,no


In [49]:
# activedefrag knobs 분리

# # activedefrag O 존재
# subTotal_knobs_Active = subTotal_knobs.dropna()
# Active_index = list(subTotal_knobs_Active.index)  # 인덱스

# # activedefrag X 존재하지 않는 나머지 반
# subTotal_knobs_NoActive = subTotal_knobs.loc[subTotal_knobs['activedefrag'].isna()].dropna(axis=1)
# NoActive_index = list(subTotal_knobs_NoActive.index)  # 인덱스

# # activedefrag O and X 전체 
# subTotal_knobs_T = subTotal_knobs.dropna(axis=1)
# total_index = list(subTotal_knobs_T.index)   # 인덱스

In [50]:
# knobs에서 타입별로 분리 (수치, 불리언, 불리언아닌 문자)

def varType(subTotal_knobs):
    object_var = []
    numerical_var = []  # 문자, 숫자 타입
    
    for idx in subTotal_knobs.dtypes.index:
        if subTotal_knobs.dtypes[idx] == 'O':  # 문자일 경우
            object_var.append(idx)
        else: 
            numerical_var.append(idx)  # 숫자일 경우

    boolean_var = []
    booleanNo_var = []  # 2개, 2개이상

    for col in object_var:
        col_count = len(subTotal_knobs[col].unique())

        if col_count == 2:  # 2개일 경우
            boolean_var.append(col)
        elif col_count > 2:  # 2개 이상일 경우
            booleanNo_var.append(col)
    
    return numerical_var, boolean_var, booleanNo_var

# 수치, 불리언, 불리언아닌것 (전체)
numerical_var, boolean_var, NobooleanNo_var = varType(subTotal_knobs_T)

# # 수치, 불리언, 불리언아닌것 (activedefrag)
# numericalActive_var, booleanActive_var, NobooleanNoActive_var = varType(subTotal_knobs_Active)

# # 수치, 불리언, 불리언아닌것 (activedefrag 없는것)
# numericalNoActive_var, booleanNoActive_var, NobooleanNoActive_var = varType(subTotal_knobs_NoActive)


In [51]:
# activedefrag O
total_internal_Active = total_internal_df.iloc[Active_index]

# activedefrag X
total_internal_NoActive = total_internal_df.iloc[NoActive_index]

# activedefrag O and X
total_internal_T = total_internal_df.iloc[total_index]

---

## 상관분석 함수

In [52]:
from scipy.stats import pearsonr
from scipy.stats import spearmanr
from scipy.stats import kendalltau


# knob하나 internal 여러개 상관계수 구하기

# pearson
## 모수 검정
def cal_pearsonr(knobs, internal_df):
    df = pd.DataFrame(columns = ['corr', 'p-value'])
    for internal in internal_df:
        result = pearsonr(knobs, internal_df[internal])
        df.loc[internal] = [result[0], result[1]]
    return df

# kendall
## 변수값 대신 순위로 바꿔서 이용, 비모수 검정
## 샘플사이저 적거나, 데이터 동률이 많을 때 유용
def cal_kendalltau(knobs, internal_df):
    df = pd.DataFrame(columns = ['corr', 'p-value'])
    for internal in internal_df:
        result = kendalltau(knobs, internal_df[internal])
        df.loc[internal] = [result[0], result[1]]
    return df

# 스피어만
## 변수값 대신 순위로 바꿔서 이용, 비모수 검정
## 데이터 내 편차와 에러에 만감하며, 일반적으로 켄달보다 높은 값을 가진다.
def cal_spearmanr(knobs, internal_df):
    df = pd.DataFrame(columns = ['corr', 'p-value'])
    for internal in internal_df:
        result = spearmanr(knobs, internal_df[internal])
        df.loc[internal] = [result[0], result[1]]
    return df


In [53]:
# knob 하나별로 3가지 상관분석 수행
def extract_pvalue(knobs, internal_df):
    print('pearson')
    df = cal_pearsonr(knobs, internal_df)
    is_pvalue = df['p-value'] <= 0.05
    print(df[is_pvalue])
    print()
    
    print('kendalltau')
    df = cal_kendalltau(knobs, internal_df)
    is_pvalue = df['p-value'] <= 0.05
    print(df[is_pvalue])
    print()
    
    print('spearman')
    df = cal_spearmanr(knobs, internal_df)
    is_pvalue = df['p-value'] <= 0.05
    print(df[is_pvalue])
    print()

# knob 리스트 각각 별로 extract_pvalue 수행
def extract_pvalue_knob(knob_list, subTotal_knobs, total_internal):
    for knob in knob_list:
        print('###{}###'.format(knob))
        extract_pvalue(subTotal_knobs[knob], total_internal)

---

## activedefrag 상관계수
* subTotal_knobs_Active
* total_internal_Active

### numericalActive_var :
* active-defrag-threshold-lower 
* active-defrag-threshold-upper 
* active-defrag-cycle-min
* active-defrag-cycle-max

In [56]:
numericalActive_var
numericalActive_var_new = numericalActive_var[6:10]
numericalActive_var_new

['active-defrag-threshold-lower',
 'active-defrag-threshold-upper',
 'active-defrag-cycle-min',
 'active-defrag-cycle-max']

In [57]:
extract_pvalue_knob(numericalActive_var_new, subTotal_knobs_Active,total_internal_Active)

###active-defrag-threshold-lower###
pearson
Empty DataFrame
Columns: [corr, p-value]
Index: []

kendalltau
Empty DataFrame
Columns: [corr, p-value]
Index: []

spearman
Empty DataFrame
Columns: [corr, p-value]
Index: []

###active-defrag-threshold-upper###
pearson
Empty DataFrame
Columns: [corr, p-value]
Index: []

kendalltau
Empty DataFrame
Columns: [corr, p-value]
Index: []

spearman
Empty DataFrame
Columns: [corr, p-value]
Index: []

###active-defrag-cycle-min###
pearson
                               corr   p-value
uptime_in_seconds         -0.079491  0.021451
instantaneous_ops_per_sec  0.074893  0.030272
instantaneous_input_kbps   0.074893  0.030271
instantaneous_output_kbps  0.074902  0.030252

kendalltau
                               corr   p-value
uptime_in_seconds         -0.095127  0.000771
used_cpu_sys              -0.049000  0.044113
instantaneous_ops_per_sec  0.052079  0.032403
instantaneous_input_kbps   0.052067  0.032443
instantaneous_output_kbps  0.052327  0.031594

spe

In [58]:
# cluster 별 knob들의 상관계수 합  
active_result = pd.DataFrame(index = numericalActive_var_new)

cluster_list = [cluster0, cluster1, cluster2, cluster3]

for idx, cluster in enumerate(cluster_list):
    cluster_val = []
    for knob in numericalActive_var_new:
        df = cal_pearsonr(subTotal_knobs_Active[knob], total_internal_Active.loc[:,cluster])
        cluster_val.append(df['corr'].abs().mean())

    active_result['cluster'+str(idx)] = cluster_val

In [59]:
# active_result

In [60]:
# active_result

---

## 전체 knob
* subTotal_knobs_T
* total_internal_T

### numerical_var

In [61]:
numerical_var

['save0_0',
 'save0_1',
 'save1_0',
 'save1_1',
 'save2_0',
 'save2_1',
 'maxmemory',
 'maxmemory-samples',
 'hash-max-ziplist-entries',
 'hash-max-ziplist-value',
 'hz']

In [62]:
extract_pvalue_knob(numerical_var, subTotal_knobs_T,total_internal_T)

###save0_0###
pearson
                       corr   p-value
uptime_in_seconds -0.048550  0.045341
used_cpu_sys      -0.050629  0.036861

kendalltau
Empty DataFrame
Columns: [corr, p-value]
Index: []

spearman
Empty DataFrame
Columns: [corr, p-value]
Index: []

###save0_1###
pearson
Empty DataFrame
Columns: [corr, p-value]
Index: []

kendalltau
                   corr   p-value
used_cpu_user -0.041212  0.027503

spearman
                   corr   p-value
used_cpu_user -0.053356  0.027816

###save1_0###
pearson
Empty DataFrame
Columns: [corr, p-value]
Index: []

kendalltau
Empty DataFrame
Columns: [corr, p-value]
Index: []

spearman
Empty DataFrame
Columns: [corr, p-value]
Index: []

###save1_1###
pearson
Empty DataFrame
Columns: [corr, p-value]
Index: []

kendalltau
                             corr   p-value
mem_fragmentation_ratio  0.039758  0.047425

spearman
                             corr   p-value
mem_fragmentation_ratio  0.048072  0.047509

###save2_0###
pearson
Empty DataFrame

In [31]:
subTotal_knobs_T  # knobs에서 aof 관련 제거한 것
total_internal_T  # 전체

,used_memory_rss,used_memory_rss_human,rss_overhead_ratio,rss_overhead_bytes,mem_fragmentation_ratio,mem_fragmentation_bytes,uptime_in_seconds,hz,lru_clock,used_memory_peak,...,used_cpu_user,allocator_allocated,allocator_active,allocator_frag_bytes,instantaneous_ops_per_sec,instantaneous_input_kbps,instantaneous_output_kbps,allocator_resident,allocator_rss_ratio,allocator_rss_bytes
0,42024960,40.08,1.00,-184320,1.09,3402352,28,11,3668563,38704616,...,4.884932,38655872,38920192,264320,16060,1121.38,79.90,42209280,1.08,3289088
1,42024960,40.08,1.00,-184320,1.09,3402352,28,34,3668604,38704632,...,4.937450,38658328,38932480,274152,28139,1964.82,139.72,42209280,1.08,3276800
2,41799680,39.86,0.99,-397312,1.08,3177072,28,3,3668639,38704616,...,4.774636,38655584,38920192,264608,39006,2723.66,193.47,42196992,1.08,3276800
3,42041344,40.09,1.00,-167936,1.09,3418736,28,26,3668673,38704616,...,5.038324,38655856,38920192,264336,16262,1135.51,81.10,42209280,1.08,3289088
4,42049536,40.10,0.95,-2260992,1.09,3426928,28,22,3668707,38704616,...,4.859807,38655744,38920192,264448,16797,1172.88,83.55,44310528,1.14,5390336
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1695,42020864,40.07,0.95,-2289664,1.09,3398256,28,22,3742679,38704616,...,4.959740,38656320,38920192,263872,22815,1593.08,113.37,44310528,1.14,5390336
1696,42016768,40.07,1.00,-192512,1.09,3394160,28,24,3742714,38704616,...,4.985621,38657928,38932480,274552,17227,1202.90,85.79,42209280,1.08,3276800
1697,42065920,40.12,0.95,-2244608,1.09,3443312,28,22,3742748,38704616,...,4.835025,38657928,38932480,274552,19414,1355.54,96.31,44310528,1.14,5378048
1698,42041344,40.09,0.95,-2269184,1.09,3336832,28,1,3742782,38704616,...,4.876490,38737672,39010304,272632,42905,2995.82,212.12,44310528,1.14,5300224


In [25]:
# cluster 별 knob들의 상관계수 합  
numerical_result = pd.DataFrame(index = numerical_var)

cluster_list = [cluster0, cluster1, cluster2, cluster3]

for idx, cluster in enumerate(cluster_list):
    cluster_val = []
    for knob in numerical_var:
        df = cal_spearmanr(subTotal_knobs_T[knob], total_internal_T.loc[:,cluster])
        cluster_val.append(df['corr'].abs().mean())

    numerical_result['cluster'+str(idx)] = cluster_val

In [26]:
numerical_result

,cluster0,cluster1,cluster2,cluster3
save0_0,0.016581,0.019750,0.015141,0.019965
save0_1,0.031612,0.018152,0.010100,0.011637
save1_0,0.008737,0.016567,0.008731,0.008228
save1_1,0.023269,0.027022,0.013835,0.008840
save2_0,0.025457,0.014749,0.007180,0.019713
save2_1,0.018830,0.016769,0.020865,0.014701
maxmemory,0.014224,0.022491,0.014605,0.008972
maxmemory-samples,0.018019,0.019869,0.017244,0.039725
hash-max-ziplist-entries,0.030972,0.017642,0.013059,0.010935
hash-max-ziplist-value,0.010711,0.013281,0.034982,0.016914


In [27]:
numerical_result

,cluster0,cluster1,cluster2,cluster3
save0_0,0.016581,0.019750,0.015141,0.019965
save0_1,0.031612,0.018152,0.010100,0.011637
save1_0,0.008737,0.016567,0.008731,0.008228
save1_1,0.023269,0.027022,0.013835,0.008840
save2_0,0.025457,0.014749,0.007180,0.019713
save2_1,0.018830,0.016769,0.020865,0.014701
maxmemory,0.014224,0.022491,0.014605,0.008972
maxmemory-samples,0.018019,0.019869,0.017244,0.039725
hash-max-ziplist-entries,0.030972,0.017642,0.013059,0.010935
hash-max-ziplist-value,0.010711,0.013281,0.034982,0.016914


---

## Activedefrag 없는 knob들
* subTotal_knobs_Active
* total_internal_Active
active인거는 신경 안써도 된다.
### numericalActive_var

In [245]:
numericalActive_var

['save0_0',
 'save0_1',
 'save1_0',
 'save1_1',
 'save2_0',
 'save2_1',
 'active-defrag-threshold-lower',
 'active-defrag-threshold-upper',
 'active-defrag-cycle-min',
 'active-defrag-cycle-max',
 'maxmemory',
 'maxmemory-samples',
 'hash-max-ziplist-entries',
 'hash-max-ziplist-value',
 'hz']

In [246]:
extract_pvalue_knob(numericalActive_var, subTotal_knobs_Active,total_internal_Active)

###save0_0###
pearson
Empty DataFrame
Columns: [corr, p-value]
Index: []

kendalltau
        corr   p-value
hz  0.037937  0.047294

spearman
Empty DataFrame
Columns: [corr, p-value]
Index: []

###save0_1###
pearson
                        corr   p-value
lru_clock           0.059351  0.035746
rdb_last_save_time  0.059352  0.035743

kendalltau
                           corr   p-value
used_memory_rss       -0.044470  0.030427
used_memory_rss_human -0.047318  0.027262
lru_clock              0.040269  0.042872
rdb_last_save_time     0.040269  0.042872

spearman
                           corr   p-value
used_memory_rss       -0.060946  0.031056
used_memory_rss_human -0.062604  0.026751
lru_clock              0.057114  0.043329
rdb_last_save_time     0.057114  0.043329

###save1_0###
pearson
                        corr   p-value
lru_clock          -0.070523  0.012561
rdb_last_save_time -0.070522  0.012563
used_cpu_sys       -0.061330  0.030011

kendalltau
                        corr   p-va

## other knob
* subTotal_knobs_NoActive
* total_internal_NoActive
active인거는 신경 안써도 된다.
### numericalNoActive_var

In [247]:
numericalNoActive_var

['save0_0',
 'save0_1',
 'save1_0',
 'save1_1',
 'save2_0',
 'save2_1',
 'maxmemory',
 'maxmemory-samples',
 'hash-max-ziplist-entries',
 'hash-max-ziplist-value',
 'hz']

In [249]:
extract_pvalue_knob(numericalNoActive_var, subTotal_knobs_NoActive,total_internal_NoActive)

###save0_0###
pearson
Empty DataFrame
Columns: [corr, p-value]
Index: []

kendalltau
Empty DataFrame
Columns: [corr, p-value]
Index: []

spearman
Empty DataFrame
Columns: [corr, p-value]
Index: []

###save0_1###
pearson
Empty DataFrame
Columns: [corr, p-value]
Index: []

kendalltau
Empty DataFrame
Columns: [corr, p-value]
Index: []

spearman
Empty DataFrame
Columns: [corr, p-value]
Index: []

###save1_0###
pearson
                         corr   p-value
lru_clock            0.058587  0.038509
rdb_last_save_time   0.058689  0.038169
allocator_resident   0.059750  0.034810
allocator_rss_ratio  0.058408  0.039104
allocator_rss_bytes  0.059935  0.034250

kendalltau
                             corr   p-value
mem_fragmentation_ratio  0.051463  0.025128
lru_clock                0.039195  0.038311
rdb_last_save_time       0.039195  0.038311
allocator_resident       0.053478  0.014550
allocator_rss_ratio      0.046319  0.045418
allocator_rss_bytes      0.047804  0.024630

spearman
            

---

## pointbiserialr 분석

In [58]:
from scipy.stats import pointbiserialr
# pointbiserialr
## yes,no 데이터와 연속형 데이터 상관분석
def cal_pointbiserialr(knobs, internal_df):
    df = pd.DataFrame(columns = ['corr', 'p-value'])
    for internal in internal_df:
        result = pointbiserialr(knobs, internal_df[internal])
        df.loc[internal] = [result[0], result[1]]
    return df

# knob 하나별로 3가지 상관분석 수행
def extract_pvalue(knobs, internal_df):
    print('pointbierialr')
    df = cal_pointbiserialr(knobs, internal_df)
    is_pvalue = df['p-value'] <= 0.05
    print(df[is_pvalue])
    print()


# knob 리스트 각각 별로 extract_pvalue 수행
def extract_pvalue_knob_point(knob_list, subTotal_knobs, total_internal):
    for knob in knob_list:
        print('###{}###'.format(knob))
        extract_pvalue(subTotal_knobs[knob], total_internal)


## other knob(activedefrag 제외)
* subTotal_knobs_T
* total_internal_T

### boolean_var

In [59]:
boolean_var

['rdbcompression',
 'rdbchecksum',
 'rdb-save-incremental-fsync',
 'lazyfree-lazy-eviction',
 'lazyfree-lazy-expire',
 'lazyfree-lazy-server-del',
 'activerehashing',
 'dynamic-hz']

In [60]:
#  yes, no -> 1, 0으로 변경
subTotal_knobs_T_bool = subTotal_knobs_T[boolean_var]  ## boolean 변수 df 추출
subTotal_knobs_T_point = subTotal_knobs_T_bool.replace(['yes', 'no'], [1,0])  ## 1, 0 변경

In [61]:
extract_pvalue_knob_point(boolean_var, subTotal_knobs_T_point, total_internal_T)

###rdbcompression###
pointbierialr
                            corr   p-value
total_net_output_bytes -0.056501  0.004715

###rdbchecksum###
pointbierialr
Empty DataFrame
Columns: [corr, p-value]
Index: []

###rdb-save-incremental-fsync###
pointbierialr
Empty DataFrame
Columns: [corr, p-value]
Index: []

###lazyfree-lazy-eviction###
pointbierialr
                         corr   p-value
rss_overhead_ratio  -0.046519  0.020017
rss_overhead_bytes  -0.048922  0.014431
allocator_resident   0.043003  0.031551
allocator_rss_ratio  0.043123  0.031080
allocator_rss_bytes  0.043044  0.031389

###lazyfree-lazy-expire###
pointbierialr
Empty DataFrame
Columns: [corr, p-value]
Index: []

###lazyfree-lazy-server-del###
pointbierialr
        corr   p-value
hz -0.045644  0.022474

###activerehashing###
pointbierialr
                         corr   p-value
allocator_allocated -0.044451  0.026248
allocator_active    -0.040469  0.043047

###dynamic-hz###
pointbierialr
                     corr   p-value
ma

In [62]:
# cluster 별 knob들의 상관계수 합
bool_result = pd.DataFrame(index = boolean_var)

cluster_list = [cluster0, cluster1, cluster2, cluster3]

for idx, cluster in enumerate(cluster_list):
    cluster_val = []
    for knob in boolean_var:
        df = cal_pointbiserialr(subTotal_knobs_T_point[knob], total_internal_T.loc[:,cluster])
        cluster_val.append(df['corr'].abs().mean())

    bool_result['cluster'+str(idx)] = cluster_val

In [63]:
bool_result

,cluster0,cluster1,cluster2,cluster3
rdbcompression,0.007174,0.024761,0.022094,0.005778
rdbchecksum,0.005528,0.011231,0.008278,0.011813
rdb-save-incremental-fsync,0.022363,0.009404,0.013930,0.017267
lazyfree-lazy-eviction,0.031083,0.011102,0.004011,0.043056
lazyfree-lazy-expire,0.020375,0.012749,0.015585,0.007907
lazyfree-lazy-server-del,0.012797,0.017518,0.008661,0.023868
activerehashing,0.021916,0.017866,0.031933,0.026625
dynamic-hz,0.014893,0.017591,0.019106,0.027647


In [314]:
bool_result

,cluster0,cluster1,cluster2,cluster3
rdbcompression,0.043045,0.247608,0.132567,0.017334
rdbchecksum,0.033166,0.112314,0.049670,0.035439
rdb-save-incremental-fsync,0.134179,0.094039,0.083578,0.051800
lazyfree-lazy-eviction,0.186498,0.111021,0.024066,0.129169
lazyfree-lazy-expire,0.122251,0.127492,0.093513,0.023721
lazyfree-lazy-server-del,0.076782,0.175183,0.051966,0.071605
activerehashing,0.131495,0.178660,0.191599,0.079875
dynamic-hz,0.089357,0.175908,0.114635,0.082941


---

In [ ]:
extract_pvalue_knob(numerical_var, subTotal_knobs_T,total_internal_T)
extract_pvalue_knob(numericalActive_var_new, subTotal_knobs_Active,total_internal_Active)
extract_pvalue_knob_point(boolean_var, subTotal_knobs_T_point, total_internal_T)